In [3]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [8]:
from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select, TextInput, Button, Paragraph
import numpy as np
import cv2

In [5]:
def modify_doc(doc):
    
    # create some widgets
    button = Button(label="Say HI")
    input = TextInput(value="Bokeh")
    output = Paragraph()

    # add a callback to a widget
    def update():
        output.text = "Hello, " + input.value
    button.on_click(update)

    # create a layout for everything
    layout = column(button, input, output)

    # add the layout to curdoc
    doc.add_root(layout)
    
# In the notebook, just pass the function that defines the app to show
# You may need to supply notebook_url, e.g notebook_url="http://localhost:8889" 


In [6]:
show(modify_doc) 

In [7]:
def get_data(N):
    return dict(x=np.random.normal(size=N), y=np.random.normal(size=N), r=np.random.normal(size=N) * 0.03)

COLORS = ["black", "firebrick", "navy", "olive", "goldenrod"]

def modify_doc(doc):
    source = ColumnDataSource(data=get_data(200))

    p = figure(tools="", toolbar_location=None)
    r = p.circle(x='x', y='y', radius='r', source=source,
                 color="navy", alpha=0.6, line_color="white")

    
    select = Select(title="Color", value="navy", options=COLORS)
    input = TextInput(title="Number of points", value="200")

    def update_color(attrname, old, new):
        r.glyph.fill_color = select.value
    select.on_change('value', update_color)

    def update_points(attrname, old, new):
        N = int(input.value)
        source.data = get_data(N)
    input.on_change('value', update_points)

    layout = column(row(select, input, width=400), row(p))

    doc.add_root(layout)

In [8]:
show(modify_doc) 

In [4]:
from bokeh.models import ColumnDataSource
import math

In [9]:

def modify_doc2(doc):
    p = figure(match_aspect=True)
    p.circle(x=0, y=0, radius=1, fill_color=None, line_width=2)
    
    # this is just to help the auto-datarange
    p.rect(0, 0, 2, 2, alpha=0)

    # this is the data source we will stream to
    source = ColumnDataSource(data=dict(x=[1], y=[0]))
    p.circle(x='x', y='y', size=12, fill_color='white', source=source)

    def update():
        x, y = source.data['x'][-1], source.data['y'][-1]

        # construct the new values for all columns, and pass to stream
        new_data = dict(x=[x*math.cos(0.1) - y*math.sin(0.1)], y=[x*math.sin(0.1) + y*math.cos(0.1)])
        source.stream(new_data, rollover=8)

    doc.add_periodic_callback(update, 150)
    doc.add_root(p)
    

In [10]:
show(modify_doc2)

-0.9352099151945389

In [42]:
def get_data(N):
    return dict(x=np.random.normal(size=N), y=np.random.normal(size=N), r=np.random.normal(size=N) * 0.03)

COLORS = ["black", "firebrick", "navy", "olive", "goldenrod"]

def modify_doc(doc):

    N = 20
    img = np.empty((N,N), dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((N, N, 4))
    for i in range(N):
        for j in range(N):
            view[i, j, 0] = int(i/N*255)
            view[i, j, 1] = 158
            view[i, j, 2] = int(j/N*255)
            view[i, j, 3] = 255
            
            

    frame=cv2.imread("./output_net.png")
    frame = cv2.flip(frame, 1)
    frameRGBA = cv2.cvtColor(frame, cv2.COLOR_RGB2RGBA)
    frameRGBA = cv2.flip(frameRGBA, 0)
    h, w, c = frameRGBA.shape
    dim = max(h, w)
    p = figure(x_range=(0,dim), y_range=(-dim,0), tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
    #p.x_range.range_padding = p.y_range.range_padding = 0
    
    # must give a vector of images
    p.image_rgba(image=[frameRGBA], x=0, y=-h, dw=w, dh=h)
    
    layout = column(p)

    doc.add_root(layout)
    

In [43]:
show(modify_doc)

1200